In [ ]:
import os 
import pandas as pd
import calendar

In [ ]:
data_file = os.path.join('./banned_books.csv')
data = pd.read_csv(data_file)

In [ ]:
# convert ID field to numbers (do I really need it)
data["ID"] = pd.to_numeric(data["ID"])

In [ ]:
# Drop columns without a lot of data
data = data.drop("Links1", axis=1)
data = data.drop("Links2", axis=1)
data = data.drop("Links3", axis=1)
data = data.drop("Links4", axis=1)
data = data.drop("Police", axis=1)
data = data.drop("Illustrators", axis=1)
data = data.drop("Book_Co_Author", axis=1)
data = data.drop("Restrictions", axis=1)
data = data.drop("Restriction_Details", axis=1)
data = data.drop("Overseeing_Agency", axis=1)
data = data.drop("Last_Updated", axis=1)
data = data.drop("Initial_Response", axis=1)
data = data.drop("Challenge_Notes", axis=1)
data = data.drop("Sch_Name", axis=1)
data = data.drop("Lib_name", axis=1)
data = data.drop("Data_Entry_Date", axis=1)
data = data.drop("Series_Name", axis=1)
data = data.drop("City", axis=1)
data = data.drop("County", axis=1)
data = data.drop("Book_Status", axis=1)

display(data.columns)

In [ ]:
# change unset values to Unclear in challenge type
data["Challenge_Type"] = data["Challenge_Type"].fillna("Unclear")

# change unset values to Affiliation Unknown in challenge init
data["Challenge_Init"] = data["Challenge_Init"].fillna("Affiliation Unknown")

# change unset values to Affiliation Unknown in challenge init
data["Decision"] = data["Decision"].fillna("Unknown")


#drop NaN from months, years, state
data = data.dropna(subset=['Month', "Year", "State"])

In [ ]:
# fix months and years
# remove weird labels 
data = data.drop(data[data["Month"] == "22-23"].index)
data = data.drop(data[data["Month"] == "Unclear"].index)
data = data.drop(data[data["Month"] == "Spring"].index)
data = data.drop(data[data["Month"] == "Summer"].index)
data = data.drop(data[data["Month"] == "Fall"].index)

# convert named months to number
months = {month: index for index, month in enumerate(calendar.month_name) if month}
for month, idx in months.items():
    data = data.replace(month, idx)

# make all months and years numeric
data["Month"] = data["Month"].astype("uint8")
data["Year"] = data["Year"].astype("uint16")

In [ ]:

# data[data["Decision"] == "Weeded/Deselected"]
data["Decision"] = data["Decision"].replace("Banned From Library", "Banned")
data["Decision"] = data["Decision"].replace("Banned/Removed", "Banned")
data["Decision"] = data["Decision"].replace("Removed", "Banned")

data["Decision"] = data["Decision"].replace("Removed - Curriculum", "Removed From Curriculum")

data["Decision"] = data["Decision"].replace("Retained", "Reshelved")
data["Decision"] = data["Decision"].replace("Returned", "Reshelved")
data["Decision"] = data["Decision"].replace("No Action", "Reshelved")

data["Decision"] = data["Decision"].replace("Challenged - Removed While Under Review", "In Process")
data["Decision"] = data["Decision"].replace("Still in Process", "In Process")
data["Decision"] = data["Decision"].replace("Still in ProcessStill in Process", "In Process")

data["Decision"] = data["Decision"].replace("Retained Restricted", "Restricted")
data["Decision"] = data["Decision"].replace("Retained/Restricted", "Restricted")

data["Decision"] = data["Decision"].replace("Weeded", "Reshelved")
data["Decision"] = data["Decision"].replace("Weeded/Deselected", "Reshelved")

# I dont know what "Lost" Means in this context and only 4 cells have the value
data = data.drop(data[data["Decision"] == '"Lost"'].index)
# data["Decision"] = data["Decision"].replace('"Lost"', "Restricted")

In [ ]:
data["Challenge_Init"] = data["Challenge_Init"].replace("School/Library Board Member", "Board Member")
data["Challenge_Init"] = data["Challenge_Init"].replace("FLCA", "Group")

In [ ]:
# recompute id field
data["author"] = data["New_Book_Author"]
data = data.drop("New_Book_Author", axis=1)

data["title"] = data["New_Book_Title"]
data = data.drop("New_Book_Title", axis=1)

data["challenge_type"] = data["Challenge_Type"]
data = data.drop("Challenge_Type", axis=1)

data["challenger"] = data["Challenge_Init"]
data = data.drop("Challenge_Init", axis=1)

data["decision"] = data["Decision"]
data = data.drop("Decision", axis=1)

data["library_type"] = data["Library_Type"]
data = data.drop("Library_Type", axis=1)

data["day"] = 1
data["month"] = data["Month"]
data = data.drop("Month", axis=1)
data["year"] = data["Year"]
data = data.drop("Year", axis=1)

data["challenge_date"] = pd.to_datetime(data[["month", "year", "day"]])

data["state"] = data["State"]
data = data.drop("State", axis=1)

In [ ]:
data.to_pickle("banned_books_clean.pickle")